In [7]:
from google.oauth2.service_account import Credentials
from sqlalchemy import create_engine
from dotenv import load_dotenv
import gspread
import pandas as pd
import os
import json

In [5]:
load_dotenv()
engine = create_engine(os.getenv('MYSQL_URI')).connect()

In [6]:
scopes = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive'
    ]

# creds_file = '../credentials.json'
creds_file = os.getenv('GOOGLE_API_CREDENTIALS')
creds = Credentials.from_service_account_info(json.loads(creds_file), scopes=scopes)
client = gspread.authorize(creds)

worksheet_map = {
    'logs': ('logs.csv', 'prod'),
    'logs-dev': ('logs.csv', 'dev'),
    'prompts': ('prompts.csv', 'prod'),
    'prompts-dev': ('prompts.csv', 'dev')
}

for table_name, (sheet_name, worksheet_name) in worksheet_map.items():
    print(f"Processing table: {table_name} -> {sheet_name}, {worksheet_name}")
    df = pd.read_sql_table(table_name, con=engine).sort_values(by='datetime', ascending=True)
    df['datetime'] = df['datetime'].astype(str)
    sheet = client.open(sheet_name)
    worksheet = sheet.worksheet(worksheet_name)
    worksheet.clear()
    worksheet.update([df.columns.values.tolist()] + df.values.tolist(), value_input_option='RAW')
    print(f"Updated {worksheet_name} in {sheet_name}")
        


Processing table: logs -> logs.csv, prod
Updated prod in logs.csv
Processing table: logs-dev -> logs.csv, dev
Updated dev in logs.csv
Processing table: prompts -> prompts.csv, prod
Updated prod in prompts.csv
Processing table: prompts-dev -> prompts.csv, dev
Updated dev in prompts.csv
